# Making the BS05(AGS,OP) solar model be a function of a one radius

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.interpolate import interp1d
import pandas as pd

Get the model files

In [2]:
main_model_name = 'BS2005-AGS,OP.csv'
neutrino_name = 'Neutrino Fluxes.csv'

model_columns = ['m','r','T','rho','P','l','X','Y','He3','C12','N14','O16']
neutrino_columns = ['r','T6','n_e','m','X 7Be','pp','B8','N13','O15','F17','Be7','pep','hep']

df_main_model = pd.read_csv(main_model_name, names=model_columns, skiprows=1,index_col=False)
df_neutrino = pd.read_csv(neutrino_name, names=neutrino_columns, skiprows=1,index_col=False)

In [3]:
df_main_model

,m,r,T,rho,P,l,X,Y,He3,C12,N14,O16
0,4.000000e-07,0.00161,15500000.0,151.000000,2.340000e+17,0.00000,0.36462,0.62029,0.000009,0.000008,0.003730,0.00606
1,5.000000e-07,0.00172,15500000.0,151.000000,2.340000e+17,0.00000,0.36462,0.62028,0.000009,0.000008,0.003730,0.00606
2,6.000000e-07,0.00182,15500000.0,151.000000,2.340000e+17,0.00001,0.36463,0.62027,0.000009,0.000008,0.003730,0.00606
3,7.000000e-07,0.00191,15500000.0,151.000000,2.340000e+17,0.00001,0.36464,0.62026,0.000009,0.000008,0.003730,0.00606
4,9.000000e-07,0.00200,15500000.0,150.000000,2.340000e+17,0.00001,0.36464,0.62025,0.000009,0.000008,0.003730,0.00606
...,...,...,...,...,...,...,...,...,...,...,...,...
1279,9.999879e-01,0.98220,86000.0,0.001050,1.140000e+10,0.99891,0.75830,0.22905,0.000101,0.002180,0.000631,0.00549
1280,9.999885e-01,0.98249,84400.0,0.001010,1.080000e+10,0.99891,0.75830,0.22905,0.000101,0.002180,0.000631,0.00549
1281,9.999888e-01,0.98261,83700.0,0.001000,1.060000e+10,0.99891,0.75830,0.22905,0.000101,0.002180,0.000631,0.00549
1282,9.999892e-01,0.98280,82700.0,0.000978,1.020000e+10,0.99891,0.75830,0.22905,0.000101,0.002180,0.000631,0.00549


In [4]:
df_neutrino

,r,T6,n_e,m,X 7Be,pp,B8,N13,O15,F17,Be7,pep,hep
0,0.00041,15.476,2.011,5.990000e-08,1.580000e-11,3.880000e-07,0.000005,2.830000e-06,0.000004,0.000004,1.910000e-06,6.070000e-07,1.660000e-07
1,0.00082,15.476,2.011,1.750000e-07,1.580000e-11,1.140000e-06,0.000014,8.280000e-06,0.000012,0.000013,5.590000e-06,1.780000e-06,4.870000e-07
2,0.00123,15.476,2.011,2.910000e-07,1.580000e-11,1.890000e-06,0.000023,1.370000e-05,0.000019,0.000021,9.280000e-06,2.950000e-06,8.070000e-07
3,0.00164,15.475,2.011,4.490000e-07,1.580000e-11,2.910000e-06,0.000036,2.110000e-05,0.000029,0.000032,1.430000e-05,4.540000e-06,1.250000e-06
4,0.00205,15.475,2.011,6.690000e-07,1.580000e-11,4.340000e-06,0.000053,3.150000e-05,0.000044,0.000048,2.130000e-05,6.780000e-06,1.860000e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,0.49795,3.905,0.071,2.940000e-04,8.770000e-24,6.010000e-08,0.000000,2.210000e-15,0.000000,0.000000,1.100000e-16,1.760000e-09,2.900000e-10
1215,0.49836,3.901,0.069,2.930000e-04,8.520000e-24,5.930000e-08,0.000000,2.140000e-15,0.000000,0.000000,1.070000e-16,1.730000e-09,2.840000e-10
1216,0.49877,3.897,0.067,2.920000e-04,8.280000e-24,5.850000e-08,0.000000,2.060000e-15,0.000000,0.000000,1.030000e-16,1.700000e-09,2.790000e-10
1217,0.49918,3.893,0.065,2.910000e-04,8.040000e-24,5.770000e-08,0.000000,1.990000e-15,0.000000,0.000000,9.920000e-17,1.670000e-09,2.730000e-10


In [5]:
true_radius = np.array(df_main_model['r'])
print(true_radius)

[0.00161 0.00172 0.00182 ... 0.98261 0.9828  0.98308]


In [6]:
def interpolate_column(A_r, B_r, B_quantity):
    interpolation_function = interp1d(B_r, B_quantity, kind='linear', fill_value=0,bounds_error=False)
    return interpolation_function(A_r)

In [7]:
df_neutrino_interpolated = pd.DataFrame()
df_neutrino_interpolated['r'] = df_main_model['r']

for column in df_neutrino.columns:
    if column != 'r':
        df_neutrino_interpolated[column] = interpolate_column(df_main_model['r'], df_neutrino['r'],df_neutrino[column])

In [8]:
df_neutrino_interpolated

,r,T6,n_e,m,X 7Be,pp,B8,N13,O15,F17,Be7,pep,hep
0,0.00161,15.475073,2.011,4.374390e-07,1.580000e-11,0.000003,0.000035,0.000021,0.000029,0.000032,0.000014,0.000004,0.000001
1,0.00172,15.475000,2.011,4.919268e-07,1.580000e-11,0.000003,0.000039,0.000023,0.000032,0.000036,0.000016,0.000005,0.000001
2,0.00182,15.475000,2.011,5.455854e-07,1.580000e-11,0.000004,0.000043,0.000026,0.000036,0.000039,0.000017,0.000006,0.000002
3,0.00191,15.475000,2.011,5.938780e-07,1.580000e-11,0.000004,0.000047,0.000028,0.000039,0.000043,0.000019,0.000006,0.000002
4,0.00200,15.475000,2.011,6.421707e-07,1.580000e-11,0.000004,0.000051,0.000030,0.000042,0.000046,0.000020,0.000007,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,0.98220,0.000000,0.000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1280,0.98249,0.000000,0.000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1281,0.98261,0.000000,0.000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1282,0.98280,0.000000,0.000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
df_out = pd.merge(df_main_model, df_neutrino_interpolated, on='r')

In [10]:
df_out

,m_x,r,T,rho,P,l,X,Y,He3,C12,...,m_y,X 7Be,pp,B8,N13,O15,F17,Be7,pep,hep
0,4.000000e-07,0.00161,15500000.0,151.000000,2.340000e+17,0.00000,0.36462,0.62029,0.000009,0.000008,...,4.374390e-07,1.580000e-11,0.000003,0.000035,0.000021,0.000029,0.000032,0.000014,0.000004,0.000001
1,5.000000e-07,0.00172,15500000.0,151.000000,2.340000e+17,0.00000,0.36462,0.62028,0.000009,0.000008,...,4.919268e-07,1.580000e-11,0.000003,0.000039,0.000023,0.000032,0.000036,0.000016,0.000005,0.000001
2,6.000000e-07,0.00182,15500000.0,151.000000,2.340000e+17,0.00001,0.36463,0.62027,0.000009,0.000008,...,5.455854e-07,1.580000e-11,0.000004,0.000043,0.000026,0.000036,0.000039,0.000017,0.000006,0.000002
3,7.000000e-07,0.00191,15500000.0,151.000000,2.340000e+17,0.00001,0.36464,0.62026,0.000009,0.000008,...,5.938780e-07,1.580000e-11,0.000004,0.000047,0.000028,0.000039,0.000043,0.000019,0.000006,0.000002
4,9.000000e-07,0.00200,15500000.0,150.000000,2.340000e+17,0.00001,0.36464,0.62025,0.000009,0.000008,...,6.421707e-07,1.580000e-11,0.000004,0.000051,0.000030,0.000042,0.000046,0.000020,0.000007,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,9.999879e-01,0.98220,86000.0,0.001050,1.140000e+10,0.99891,0.75830,0.22905,0.000101,0.002180,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1280,9.999885e-01,0.98249,84400.0,0.001010,1.080000e+10,0.99891,0.75830,0.22905,0.000101,0.002180,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1281,9.999888e-01,0.98261,83700.0,0.001000,1.060000e+10,0.99891,0.75830,0.22905,0.000101,0.002180,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1282,9.999892e-01,0.98280,82700.0,0.000978,1.020000e+10,0.99891,0.75830,0.22905,0.000101,0.002180,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
df_out.to_csv('interpolated_model.csv',index=False)